## Fitting Induction data to the Transitional Matrix model using softmax rule

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tesser import cython_sr
from tesser import cython_fit
from tesser import util
import importlib

In [2]:
data_dir= "/mnt/c/Users/rodv2/Dropbox/tesser_successor/Data"
tdir = "/home/work-machine/Preston_Lab/tesser_successor/Data"

In [3]:
induct_all = util.load_induct(data_dir)
struct_all = util.load_struct(data_dir)
n_states = 21
subj_list = util.subj_list()

In [4]:
transition_all = cython_sr.transition_multiple(subj_list, tdir+"/Transitional_Matrices")

In [5]:
transition_all.keys()

dict_keys([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138])

# Transitional matrix only

In [6]:
tranmodel = transition_all

In [7]:
# Using a hybrid model function when weight 'w' is set to 0 the model is fixed only to the community matrix.
fixed = {'w': 0}
var_names = ['gamma', 'alpha', 'tau']
var_bounds = {'gamma': [0, 1], 'alpha': [0, 1], 'tau': [0, 10]}
results = cython_fit.fit_induct_indiv(struct_all, induct_all, fixed, var_names, var_bounds, split =False,
                               n_states=n_states, verbose=False, model_type='true transitional', model=tranmodel, split_list =[])

Estimating parameters for 100...


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

In [ ]:
results = results.set_index('subject')
results.head()

## Get response probabilities for the fitted models

In [ ]:
importlib.reload(cython_fit)

In [ ]:
res_list = []
for subject in results.index.unique():
    subj_filter = f'SubjNum == {subject}'
    subj_struct = struct_all.query(subj_filter)
    subj_induct = induct_all.query(subj_filter)
    subj_param = results.loc[subject]
    param = {'alpha': subj_param['alpha'], 'gamma': subj_param['gamma'],
             'tau': subj_param['tau'], 'w' : subj_param['w']}
    res = cython_fit.assess_induct_fit_subject_hybrid(subj_struct, subj_induct, param, n_states, False, model=tranmodel)
    res_list.append(res)
fitted = pd.concat(res_list, axis=0)

In [ ]:
fitted

## Plot individual differences in induction

In [ ]:
# m = fitted.groupby(['Source', 'SubjNum', 'Environment'])['Accuracy'].mean().unstack(level=0)
# g = sns.scatterplot(x='Model', y='Data', hue='Environment', data=m.reset_index());
# g.set_xlim(0, 1.02);
# g.set_ylim(0, 1.02);
# g.set_aspect(1);
# g.plot((0, 1), (0, 1), '-k');

In [ ]:
# m = fitted.groupby(['Source', 'SubjNum', 'QuestType'])['Accuracy'].mean().unstack(level=0)
# g = sns.scatterplot(x='Model', y='Data', hue='QuestType', data=m.reset_index());
# g.set_xlim(0, 1.02);
# g.set_ylim(0, 1.02);
# g.set_aspect(1);
# g.plot((0, 1), (0, 1), '-k');

## Plot group means

In [ ]:
m = fitted.groupby(['Source', 'SubjNum', 'Environment'])['Accuracy'].mean()
g = sns.catplot(kind='point', x='Environment', y='Accuracy', 
                hue='Source', dodge=True, data=m.reset_index())
g.set(ylim=(0, 1.02));

In [ ]:
m = fitted.groupby(['Source', 'SubjNum', 'QuestType'])['Accuracy'].mean()
g = sns.catplot(kind='point', x='QuestType', y='Accuracy', 
                hue='Source', dodge=True, data=m.reset_index())
g.set(ylim=(0, 1.02));

In [ ]:
results.to_csv("./Data/results/transition_only.csv")